<a href="https://colab.research.google.com/github/lmencisoe/ADL/blob/main/Project/Limpieza_datos_Proj_ADL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from pandas_profiling import ProfileReport
import tensorflow as tf
from sklearn.feature_extraction import DictVectorizer
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from xgboost import plot_importance
from keras.layers import Dropout, Flatten, Dense, Input, Rescaling, Lambda, MaxPooling2D, Conv2D, AveragePooling2D
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection  import GridSearchCV
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.utils.vis_utils import plot_model

import statistics
from matplotlib import pyplot
import seaborn as sns
import shap
import warnings
import re
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [3]:
df=pd.read_csv("/content/drive/MyDrive/Proyecto_ADL/int_matches_adl.csv",sep=";")

In [4]:
df.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,tournament,neutral_location,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,3/09/2004,Maldives,Oman,Africa,Asia,142,50,0,0,Friendly,False,Lose,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3/09/2004,Netherlands,Liechtenstein,Europe,Europe,6,152,0,0,Friendly,False,Win,93.0,72.0,85.0,93.0,87.0,NaN,NaN,NaN
2,3/09/2004,Spain,Scotland,Europe,Europe,3,67,0,0,Friendly,False,Draw,94.0,84.0,86.5,89.3,89.5,80.2,79.7,81.8
3,4/09/2004,Albania,Greece,Europe,Europe,94,14,0,0,FIFA World Cup qualification,False,Win,NaN,90.0,NaN,59.0,74.5,82.8,83.7,85.2
4,4/09/2004,Austria,England,Europe,Europe,90,7,0,0,FIFA World Cup qualification,False,Draw,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2


In [5]:
df2 = df.copy()

In [6]:
conditions = [
    (df2['home_team_result'] == 'Win'),
    (df2['home_team_result'] == 'Draw'),
    (df2['home_team_result'] == 'Lose')
    ]
values = [1, 2, 3]

In [7]:
df2['results'] = np.select(conditions, values)

In [8]:
df2 = df2.drop(['home_team_result'], axis=1)

In [9]:
df2.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,tournament,neutral_location,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,results
0,3/09/2004,Maldives,Oman,Africa,Asia,142,50,0,0,Friendly,False,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,3
1,3/09/2004,Netherlands,Liechtenstein,Europe,Europe,6,152,0,0,Friendly,False,93.0,72.0,85.0,93.0,87.0,NaN,NaN,NaN,1
2,3/09/2004,Spain,Scotland,Europe,Europe,3,67,0,0,Friendly,False,94.0,84.0,86.5,89.3,89.5,80.2,79.7,81.8,2
3,4/09/2004,Albania,Greece,Europe,Europe,94,14,0,0,FIFA World Cup qualification,False,NaN,90.0,NaN,59.0,74.5,82.8,83.7,85.2,1
4,4/09/2004,Austria,England,Europe,Europe,90,7,0,0,FIFA World Cup qualification,False,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2,2


In [10]:
conditions = [
    (df2['tournament'] == 'Friendly'),
    (df2['tournament'] != 'Friendly')
    ]
values = [0, 1]
df2['official_t'] = np.select(conditions, values)
df2.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,tournament,...,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,results,official_t
0,3/09/2004,Maldives,Oman,Africa,Asia,142,50,0,0,Friendly,...,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,3,0
1,3/09/2004,Netherlands,Liechtenstein,Europe,Europe,6,152,0,0,Friendly,...,93.0,72.0,85.0,93.0,87.0,NaN,NaN,NaN,1,0
2,3/09/2004,Spain,Scotland,Europe,Europe,3,67,0,0,Friendly,...,94.0,84.0,86.5,89.3,89.5,80.2,79.7,81.8,2,0
3,4/09/2004,Albania,Greece,Europe,Europe,94,14,0,0,FIFA World Cup qualification,...,NaN,90.0,NaN,59.0,74.5,82.8,83.7,85.2,1,1
4,4/09/2004,Austria,England,Europe,Europe,90,7,0,0,FIFA World Cup qualification,...,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2,2,1


In [11]:
df2 = df2.drop(['tournament'], axis=1)

In [12]:
df2 = df2.drop(['home_team_total_fifa_points', 'away_team_total_fifa_points'], axis=1)

In [13]:
df2['date'] = pd.to_datetime(df2['date']).dt.date

In [14]:
df2.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,neutral_location,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,results,official_t
0,2004-03-09,Maldives,Oman,Africa,Asia,142,50,False,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,3,0
1,2004-03-09,Netherlands,Liechtenstein,Europe,Europe,6,152,False,93.0,72.0,85.0,93.0,87.0,NaN,NaN,NaN,1,0
2,2004-03-09,Spain,Scotland,Europe,Europe,3,67,False,94.0,84.0,86.5,89.3,89.5,80.2,79.7,81.8,2,0
3,2004-04-09,Albania,Greece,Europe,Europe,94,14,False,NaN,90.0,NaN,59.0,74.5,82.8,83.7,85.2,1,1
4,2004-04-09,Austria,England,Europe,Europe,90,7,False,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2,2,1


In [15]:
df2['year'] = pd.DatetimeIndex(df2['date']).year

In [16]:
df2.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,neutral_location,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,results,official_t,year
0,2004-03-09,Maldives,Oman,Africa,Asia,142,50,False,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,3,0,2004
1,2004-03-09,Netherlands,Liechtenstein,Europe,Europe,6,152,False,93.0,72.0,85.0,93.0,87.0,NaN,NaN,NaN,1,0,2004
2,2004-03-09,Spain,Scotland,Europe,Europe,3,67,False,94.0,84.0,86.5,89.3,89.5,80.2,79.7,81.8,2,0,2004
3,2004-04-09,Albania,Greece,Europe,Europe,94,14,False,NaN,90.0,NaN,59.0,74.5,82.8,83.7,85.2,1,1,2004
4,2004-04-09,Austria,England,Europe,Europe,90,7,False,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2,2,1,2004


In [17]:
cuts = 4
df2['home_q'] = pd.qcut(df2['home_team_fifa_rank'],q=cuts, labels=range(1, cuts+1))
df2['away_q'] = pd.qcut(df2['away_team_fifa_rank'],q=cuts, labels=range(1, cuts+1))

In [18]:
df2.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,neutral_location,home_team_goalkeeper_score,away_team_goalkeeper_score,...,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,results,official_t,year,home_q,away_q
0,2004-03-09,Maldives,Oman,Africa,Asia,142,50,False,NaN,70.0,...,NaN,NaN,NaN,NaN,NaN,3,0,2004,4,2
1,2004-03-09,Netherlands,Liechtenstein,Europe,Europe,6,152,False,93.0,72.0,...,93.0,87.0,NaN,NaN,NaN,1,0,2004,1,4
2,2004-03-09,Spain,Scotland,Europe,Europe,3,67,False,94.0,84.0,...,89.3,89.5,80.2,79.7,81.8,2,0,2004,1,2
3,2004-04-09,Albania,Greece,Europe,Europe,94,14,False,NaN,90.0,...,59.0,74.5,82.8,83.7,85.2,1,1,2004,3,1
4,2004-04-09,Austria,England,Europe,Europe,90,7,False,83.0,88.0,...,73.0,74.0,90.5,88.7,91.2,2,1,2004,3,1


In [19]:
df2_home = df2.groupby(['year', 'home_q'])['home_team_goalkeeper_score', 'home_team_mean_defense_score', 'home_team_mean_offense_score', 'home_team_mean_midfield_score'].median()
df2_away = df2.groupby(['year', 'away_q'])['away_team_goalkeeper_score', 'away_team_mean_defense_score', 'away_team_mean_offense_score', 'away_team_mean_midfield_score'].median()

In [20]:
df2_away = pd.DataFrame(df2_away)
df2_away = df2_away.rename(columns={"away_team_goalkeeper_score": "a", "away_team_mean_defense_score": "b", "away_team_mean_offense_score": "c", "away_team_mean_midfield_score": "d"})
df2_home = pd.DataFrame(df2_home)
df2_home = df2_home.rename(columns={"home_team_goalkeeper_score": "a1", "home_team_mean_defense_score": "b1", "home_team_mean_offense_score": "c1", "home_team_mean_midfield_score": "d1"})

In [21]:
new_df = pd.merge(df2, df2_home,  how='left', left_on=['year','home_q'], right_on = ['year','home_q'])
new_df2 = pd.merge(new_df, df2_away,  how='left', left_on=['year','away_q'], right_on = ['year','away_q'])

In [22]:
new_df2

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,neutral_location,home_team_goalkeeper_score,away_team_goalkeeper_score,...,home_q,away_q,a1,b1,c1,d1,a,b,c,d
0,2004-03-09,Maldives,Oman,Africa,Asia,142,50,False,NaN,70.0,...,4,2,72.0,NaN,71.7,NaN,71.0,71.20,76.35,73.5
1,2004-03-09,Netherlands,Liechtenstein,Europe,Europe,6,152,False,93.0,72.0,...,1,4,82.0,79.5,80.3,82.0,72.0,NaN,NaN,NaN
2,2004-03-09,Spain,Scotland,Europe,Europe,3,67,False,94.0,84.0,...,1,2,82.0,79.5,80.3,82.0,71.0,71.20,76.35,73.5
3,2004-04-09,Albania,Greece,Europe,Europe,94,14,False,NaN,90.0,...,3,1,78.0,71.8,73.0,69.2,80.0,79.50,80.30,79.9
4,2004-04-09,Austria,England,Europe,Europe,90,7,False,83.0,88.0,...,3,1,78.0,71.8,73.0,69.2,80.0,79.50,80.30,79.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,2022-06-14,Moldova,Andorra,Europe,Europe,180,153,False,65.0,NaN,...,4,4,65.0,71.5,68.3,68.0,65.0,69.85,64.00,66.8
15655,2022-06-14,Liechtenstein,Latvia,Europe,Europe,192,135,False,NaN,65.0,...,4,4,65.0,71.5,68.3,68.0,65.0,69.85,64.00,66.8
15656,2022-06-14,Chile,Ghana,South America,Africa,28,60,True,79.0,74.0,...,1,2,80.0,77.8,80.7,78.5,73.0,73.00,74.00,74.2
15657,2022-06-14,Japan,Tunisia,Asia,Africa,23,35,False,73.0,NaN,...,1,1,80.0,77.8,80.7,78.5,80.0,77.80,80.00,78.5


In [23]:
new_df2['home_team_goalkeeper_score'][new_df2['home_team_goalkeeper_score'].isnull()] = new_df2['a1'][new_df2['home_team_goalkeeper_score'].isnull()]
new_df2['home_team_mean_defense_score'][new_df2['home_team_mean_defense_score'].isnull()] = new_df2['b1'][new_df2['home_team_mean_defense_score'].isnull()]
new_df2['home_team_mean_offense_score'][new_df2['home_team_mean_offense_score'].isnull()] = new_df2['c1'][new_df2['home_team_mean_offense_score'].isnull()]
new_df2['home_team_mean_midfield_score'][new_df2['home_team_mean_midfield_score'].isnull()] = new_df2['d1'][new_df2['home_team_mean_midfield_score'].isnull()]
new_df2['away_team_goalkeeper_score'][new_df2['away_team_goalkeeper_score'].isnull()] = new_df2['a'][new_df2['away_team_goalkeeper_score'].isnull()]
new_df2['away_team_mean_defense_score'][new_df2['away_team_mean_defense_score'].isnull()] = new_df2['b'][new_df2['away_team_mean_defense_score'].isnull()]
new_df2['away_team_mean_offense_score'][new_df2['away_team_mean_offense_score'].isnull()] = new_df2['c'][new_df2['away_team_mean_offense_score'].isnull()]
new_df2['away_team_mean_midfield_score'][new_df2['away_team_mean_midfield_score'].isnull()] = new_df2['d'][new_df2['away_team_mean_midfield_score'].isnull()]
new_df2

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,neutral_location,home_team_goalkeeper_score,away_team_goalkeeper_score,...,home_q,away_q,a1,b1,c1,d1,a,b,c,d
0,2004-03-09,Maldives,Oman,Africa,Asia,142,50,False,72.0,70.0,...,4,2,72.0,NaN,71.7,NaN,71.0,71.20,76.35,73.5
1,2004-03-09,Netherlands,Liechtenstein,Europe,Europe,6,152,False,93.0,72.0,...,1,4,82.0,79.5,80.3,82.0,72.0,NaN,NaN,NaN
2,2004-03-09,Spain,Scotland,Europe,Europe,3,67,False,94.0,84.0,...,1,2,82.0,79.5,80.3,82.0,71.0,71.20,76.35,73.5
3,2004-04-09,Albania,Greece,Europe,Europe,94,14,False,78.0,90.0,...,3,1,78.0,71.8,73.0,69.2,80.0,79.50,80.30,79.9
4,2004-04-09,Austria,England,Europe,Europe,90,7,False,83.0,88.0,...,3,1,78.0,71.8,73.0,69.2,80.0,79.50,80.30,79.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,2022-06-14,Moldova,Andorra,Europe,Europe,180,153,False,65.0,65.0,...,4,4,65.0,71.5,68.3,68.0,65.0,69.85,64.00,66.8
15655,2022-06-14,Liechtenstein,Latvia,Europe,Europe,192,135,False,65.0,65.0,...,4,4,65.0,71.5,68.3,68.0,65.0,69.85,64.00,66.8
15656,2022-06-14,Chile,Ghana,South America,Africa,28,60,True,79.0,74.0,...,1,2,80.0,77.8,80.7,78.5,73.0,73.00,74.00,74.2
15657,2022-06-14,Japan,Tunisia,Asia,Africa,23,35,False,73.0,80.0,...,1,1,80.0,77.8,80.7,78.5,80.0,77.80,80.00,78.5


In [24]:
new_df2 = new_df2.drop(['a', 'b', 'c', 'd', 'a1', 'b1', 'c1', 'd1', 'year', 'home_q', 	'away_q'], axis=1)

In [25]:
new_df2.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,neutral_location,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,results,official_t
0,2004-03-09,Maldives,Oman,Africa,Asia,142,50,False,72.0,70.0,NaN,71.7,NaN,71.2,76.35,73.5,3,0
1,2004-03-09,Netherlands,Liechtenstein,Europe,Europe,6,152,False,93.0,72.0,85.0,93.0,87.0,NaN,NaN,NaN,1,0
2,2004-03-09,Spain,Scotland,Europe,Europe,3,67,False,94.0,84.0,86.5,89.3,89.5,80.2,79.70,81.8,2,0
3,2004-04-09,Albania,Greece,Europe,Europe,94,14,False,78.0,90.0,71.8,59.0,74.5,82.8,83.70,85.2,1,1
4,2004-04-09,Austria,England,Europe,Europe,90,7,False,83.0,88.0,76.2,73.0,74.0,90.5,88.70,91.2,2,1


In [26]:
# home_team_goalkeeper_score	
home1_median = statistics.median(new_df2['home_team_goalkeeper_score'])
new_df2['home_team_goalkeeper_score'][pd.isna(new_df2['home_team_goalkeeper_score'])] = home1_median
#	home_team_mean_defense_score
home2_median = statistics.median(new_df2['home_team_mean_defense_score'])
new_df2['home_team_mean_defense_score'][pd.isna(new_df2['home_team_mean_defense_score'])] = home2_median
#	home_team_mean_offense_score
home3_median = statistics.median(new_df2['home_team_mean_offense_score'])
new_df2['home_team_mean_offense_score'][pd.isna(new_df2['home_team_mean_offense_score'])] = home3_median
#	home_team_mean_midfield_score
home4_median = statistics.median(new_df2['home_team_mean_midfield_score'])
new_df2['home_team_mean_midfield_score'][pd.isna(new_df2['home_team_mean_midfield_score'])] = home4_median
#	away_team_goalkeeper_score	
away1_median = statistics.median(new_df2['away_team_goalkeeper_score'])
new_df2['away_team_goalkeeper_score'][pd.isna(new_df2['away_team_goalkeeper_score'])] = away1_median
#	away_team_mean_defense_score
away2_median = statistics.median(new_df2['away_team_mean_defense_score'])
new_df2['away_team_mean_defense_score'][pd.isna(new_df2['away_team_mean_defense_score'])] = away2_median
#	away_team_mean_offense_score
away3_median = statistics.median(new_df2['away_team_mean_offense_score'])
new_df2['away_team_mean_offense_score'][pd.isna(new_df2['away_team_mean_offense_score'])] = away3_median
#	away_team_mean_midfield_score	
away4_median = statistics.median(new_df2['away_team_mean_midfield_score'])
new_df2['away_team_mean_midfield_score'][pd.isna(new_df2['away_team_mean_midfield_score'])] = away4_median

In [27]:
new_df2

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,neutral_location,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,results,official_t
0,2004-03-09,Maldives,Oman,Africa,Asia,142,50,False,72.0,70.0,61.65,71.7,68.0,71.20,76.35,73.5,3,0
1,2004-03-09,Netherlands,Liechtenstein,Europe,Europe,6,152,False,93.0,72.0,85.00,93.0,87.0,63.20,65.50,68.5,1,0
2,2004-03-09,Spain,Scotland,Europe,Europe,3,67,False,94.0,84.0,86.50,89.3,89.5,80.20,79.70,81.8,2,0
3,2004-04-09,Albania,Greece,Europe,Europe,94,14,False,78.0,90.0,71.80,59.0,74.5,82.80,83.70,85.2,1,1
4,2004-04-09,Austria,England,Europe,Europe,90,7,False,83.0,88.0,76.20,73.0,74.0,90.50,88.70,91.2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,2022-06-14,Moldova,Andorra,Europe,Europe,180,153,False,65.0,65.0,71.50,68.3,68.0,69.85,64.00,66.8,1,1
15655,2022-06-14,Liechtenstein,Latvia,Europe,Europe,192,135,False,65.0,65.0,71.50,68.3,68.0,69.85,64.00,66.8,3,1
15656,2022-06-14,Chile,Ghana,South America,Africa,28,60,True,79.0,74.0,75.50,76.7,78.2,75.50,76.00,78.2,3,1
15657,2022-06-14,Japan,Tunisia,Asia,Africa,23,35,False,73.0,80.0,75.20,75.0,77.5,70.80,72.30,74.0,3,1


In [28]:
home_cont = pd.get_dummies(new_df2['home_team_continent'], prefix='home_team_continent_').reset_index(drop=True)
away_cont = pd.get_dummies(new_df2['away_team_continent'], prefix='away_team_continent_').reset_index(drop=True)

In [29]:
new_df2 = pd.concat([new_df2.reset_index(drop=True), home_cont], axis=1)
new_df2 = pd.concat([new_df2.reset_index(drop=True), away_cont], axis=1)

In [30]:
new_df2.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,neutral_location,home_team_goalkeeper_score,away_team_goalkeeper_score,...,home_team_continent__Europe,home_team_continent__North America,home_team_continent__Oceania,home_team_continent__South America,away_team_continent__Africa,away_team_continent__Asia,away_team_continent__Europe,away_team_continent__North America,away_team_continent__Oceania,away_team_continent__South America
0,2004-03-09,Maldives,Oman,Africa,Asia,142,50,False,72.0,70.0,...,0,0,0,0,0,1,0,0,0,0
1,2004-03-09,Netherlands,Liechtenstein,Europe,Europe,6,152,False,93.0,72.0,...,1,0,0,0,0,0,1,0,0,0
2,2004-03-09,Spain,Scotland,Europe,Europe,3,67,False,94.0,84.0,...,1,0,0,0,0,0,1,0,0,0
3,2004-04-09,Albania,Greece,Europe,Europe,94,14,False,78.0,90.0,...,1,0,0,0,0,0,1,0,0,0
4,2004-04-09,Austria,England,Europe,Europe,90,7,False,83.0,88.0,...,1,0,0,0,0,0,1,0,0,0


In [113]:
new_df3 = new_df2.copy()

In [114]:
new_df3[['home_team_fifa_rank',
         'away_team_fifa_rank',
         'home_team_goalkeeper_score',
         'away_team_goalkeeper_score',
         'home_team_mean_defense_score',
         'home_team_mean_offense_score',
         'home_team_mean_midfield_score',
         'away_team_mean_defense_score',
         'away_team_mean_offense_score',
         'away_team_mean_midfield_score']] = StandardScaler().fit_transform(new_df3[['home_team_fifa_rank',
         'away_team_fifa_rank',
         'home_team_goalkeeper_score',
         'away_team_goalkeeper_score',
         'home_team_mean_defense_score',
         'home_team_mean_offense_score',
         'home_team_mean_midfield_score',
         'away_team_mean_defense_score',
         'away_team_mean_offense_score',
         'away_team_mean_midfield_score']])

In [115]:
home_max = new_df3.groupby(['home_team'], as_index=False)['date'].max()
home_max = pd.DataFrame(home_max)
away_max = new_df3.groupby(['away_team'], as_index=False)['date'].max()
away_max = pd.DataFrame(away_max)

In [116]:
home_max = home_max.rename(columns={"home_team": "team"})
away_max = away_max.rename(columns={"away_team": "team"})

In [117]:
union_dfs = pd.concat([home_max, away_max])
union_dfs = union_dfs.drop_duplicates()

In [118]:
union_dfs = union_dfs.groupby(['team'], as_index=False)['date'].max()
union_dfs = pd.DataFrame(union_dfs)

,team,date
0,Afghanistan,2022-11-06
1,Albania,2022-10-06
2,Algeria,2022-12-06
3,American Samoa,2019-10-07
4,Andorra,2022-10-06
...,...,...
207,Vietnam,2022-03-29
208,Wales,2022-11-06
209,Yemen,2022-11-06
210,Zambia,2022-07-06


In [119]:
home_info_total = new_df3[['home_team', 'date', 'home_team_fifa_rank', 'home_team_goalkeeper_score', 'home_team_mean_defense_score', 'home_team_mean_offense_score', 'home_team_mean_midfield_score', 'home_team_continent__Africa', 'home_team_continent__Asia', 'home_team_continent__Europe', 'home_team_continent__North America', 'home_team_continent__Oceania', 'home_team_continent__South America']]
away_info_total = new_df3[['away_team', 'date', 'away_team_fifa_rank', 'away_team_goalkeeper_score', 'away_team_mean_defense_score', 'away_team_mean_offense_score', 'away_team_mean_midfield_score', 'away_team_continent__Africa', 'away_team_continent__Asia', 'away_team_continent__Europe', 'away_team_continent__North America', 'away_team_continent__Oceania', 'away_team_continent__South America']]

In [120]:
home_info_total = home_info_total.rename(columns={"home_team": "team",
                                                  "home_team_fifa_rank": "team1_rank",
                                                  "home_team_goalkeeper_score": "team1_gk",
                                                  "home_team_mean_defense_score": "team1_de",
                                                  "home_team_mean_offense_score": "team1_of",
                                                  "home_team_mean_midfield_score": "team1_mi", 
                                                  "home_team_continent__Africa": "team1_af",
                                                  "home_team_continent__Asia": "team1_as",
                                                  "home_team_continent__Europe": "team1_eu",
                                                  "home_team_continent__North America": "team1_na",
                                                  "home_team_continent__Oceania": "team1_oc",
                                                  "home_team_continent__South America": "team1_sa"})
away_info_total = away_info_total.rename(columns={"away_team": "team",
                                                  "away_team_fifa_rank": "team1_rank",
                                                  "away_team_goalkeeper_score": "team1_gk",
                                                  "away_team_mean_defense_score": "team1_de",
                                                  "away_team_mean_offense_score": "team1_of",
                                                  "away_team_mean_midfield_score": "team1_mi", 
                                                  "away_team_continent__Africa": "team1_af",
                                                  "away_team_continent__Asia": "team1_as",
                                                  "away_team_continent__Europe": "team1_eu",
                                                  "away_team_continent__North America": "team1_na",
                                                  "away_team_continent__Oceania": "team1_oc",
                                                  "away_team_continent__South America": "team1_sa"})

In [121]:
union_dfs_info = pd.concat([home_info_total, away_info_total])
union_dfs_info = union_dfs_info.drop_duplicates()

,team,date,team1_rank,team1_gk,team1_de,team1_of,team1_mi,team1_af,team1_as,team1_eu,team1_na,team1_oc,team1_sa
0,Maldives,2004-03-09,1.126734,0.023561,-1.605594,-0.163232,-0.688680,1,0,0,0,0,0
1,Netherlands,2004-03-09,-1.392090,2.841912,2.218586,3.213115,2.282443,0,0,1,0,0,0
2,Spain,2004-03-09,-1.447652,2.976119,2.464250,2.626614,2.673380,0,0,1,0,0,0
3,Albania,2004-04-09,0.237737,0.828804,0.056737,-2.176360,0.327757,0,0,1,0,0,0
4,Austria,2004-04-09,0.163654,1.499840,0.777353,0.042836,0.249569,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,Andorra,2022-06-14,1.270454,-0.863433,-0.231977,-1.328384,-0.823001,0,0,1,0,0,0
15655,Latvia,2022-06-14,0.938614,-0.863433,-0.231977,-1.328384,-0.823001,0,0,1,0,0,0
15656,Ghana,2022-06-14,-0.444055,0.386016,0.768603,0.606344,1.024341,1,0,0,0,0,0
15657,Tunisia,2022-06-14,-0.904945,1.218982,-0.063737,0.009803,0.343741,1,0,0,0,0,0


In [122]:
base_total_max = pd.merge(union_dfs, union_dfs_info,  how='left', left_on=['team','date'], right_on = ['team','date'])

,team,date,team1_rank,team1_gk,team1_de,team1_of,team1_mi,team1_af,team1_as,team1_eu,team1_na,team1_oc,team1_sa
0,Afghanistan,2022-11-06,1.215148,-0.863433,-0.231977,-1.328384,-0.823001,0,1,0,0,0,0
1,Albania,2022-10-06,-0.280844,1.097218,0.777353,-0.432706,0.093194,0,0,1,0,0,0
2,Algeria,2022-12-06,-0.739025,0.941327,1.211338,1.412481,0.991931,1,0,0,0,0,0
3,American Samoa,2019-10-07,2.015731,-0.379061,-0.729390,-0.226638,-0.688680,0,0,0,0,1,0
4,Andorra,2022-10-06,1.330463,-0.915890,0.007604,-0.702180,-0.688680,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Vietnam,2022-03-29,0.256497,-1.002261,-0.471053,-0.522247,-0.336858,0,1,0,0,0,0
208,Wales,2022-11-06,-1.169841,0.291975,0.580821,0.042836,0.953256,0,0,1,0,0,0
209,Yemen,2022-11-06,1.293421,-0.915890,0.007604,-0.702180,-0.688680,0,1,0,0,0,0
210,Zambia,2022-07-06,0.108092,-0.781683,-0.434593,-0.163232,-0.407205,1,0,0,0,0,0


In [123]:
new_df3 = new_df3.drop(columns=['date',	'home_team_continent',	'away_team_continent'])

In [125]:
new_df3 = new_df3.rename(columns={"home_team": "team",
                                                  "home_team_fifa_rank": "team1_rank",
                                                  "home_team_goalkeeper_score": "team1_gk",
                                                  "home_team_mean_defense_score": "team1_de",
                                                  "home_team_mean_offense_score": "team1_of",
                                                  "home_team_mean_midfield_score": "team1_mi", 
                                                  "home_team_continent__Africa": "team1_af",
                                                  "home_team_continent__Asia": "team1_as",
                                                  "home_team_continent__Europe": "team1_eu",
                                                  "home_team_continent__North America": "team1_na",
                                                  "home_team_continent__Oceania": "team1_oc",
                                                  "home_team_continent__South America": "team1_sa",
                                                  "away_team": "team2",
                                                  "away_team_fifa_rank": "team2_rank",
                                                  "away_team_goalkeeper_score": "team2_gk",
                                                  "away_team_mean_defense_score": "team2_de",
                                                  "away_team_mean_offense_score": "team2_of",
                                                  "away_team_mean_midfield_score": "team2_mi", 
                                                  "away_team_continent__Africa": "team2_af",
                                                  "away_team_continent__Asia": "team2_as",
                                                  "away_team_continent__Europe": "team2_eu",
                                                  "away_team_continent__North America": "team2_na",
                                                  "away_team_continent__Oceania": "team2_oc",
                                                  "away_team_continent__South America": "team2_sa"})

In [179]:
new_df3.to_csv('/content/drive/MyDrive/Proyecto_ADL/data_modelo.csv', sep=';' ,index=False)
base_total_max.to_csv('/content/drive/MyDrive/Proyecto_ADL/equipos.csv', sep=';' ,index=False)